# Preview

## Issues

todo
- try to see if i can nn the tot, expecting large variance for that unknown tail

ideas
- transform of outputs
- pca outputs, to reduce dimensionality as needed
- pca inputs, to reduce complexity
- test for signal
  - 1 vs 2 nearest neighbor outputs
  - chl-a retrieval algorithms
- dealing with unbalanced data (are they unbalanced?)
- try classification only

## Imports

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
import importlib
import warnings
import datetime as dt

from IPython.display import Markdown
from scipy.stats import zscore
import holoviews as hv
import hvplot.xarray
import numpy as np
import pandas as pd
import panel as pn
import param as p
import xarray as xr

# import tensorflow as tf
# import tensorflow_datasets as tfds
# import tensorflow_probability as tfp

# from re_nobm_pcc import preprocess
from re_nobm_pcc import DATADIR, TAXA, WAVELENGTH
from re_nobm_pcc import kit

warnings.filterwarnings(action="ignore", category=FutureWarning)
hv.opts.defaults(
    hv.opts.Curve(active_tools=[]),
    hv.opts.Image(active_tools=[]),
    hv.opts.Scatter(active_tools=[]),
    hv.opts.HexTiles(active_tools=[], tools=["hover"]),
)

## Abbreviations

In [3]:
long_name = {
    "alk": "alkalinity",
    "cdc": "colored dissolved carbon",
    "chl": "chlorophytes",
    "coc": "coccolithophores",
    "cya": "cyanobacteria",
    "dia": "diatoms",
    "dic": "dissolved organic carbon",
    "din": "dinoflagellate",
    "doc": "dissolved organic carbon",
    "dtc": "dissolved total carbon",
    "fco": "carbon dioxide flux",
    "h": "mixed layer depth",
    "irn": "iron",
    "pco": "carbon dioxide concentration",
    "pha": "phaeocystis",
    "pic": "particulate inorganic carbon",
    "pp": "phytoplankton primary productivity",
    "tpp": "total primary productivity",
    "rnh": "ammonium",
    "rno": "nitrate",
    "s": "salinity",
    "t": "temperature",
    "tot": "total chlorophyl",
    "zoo": "zooplankton",
}

## Raw Data

The OASIM model requires absorption and backscattering for each phytoplankton group.

In [4]:
ds = []
for item in TAXA:
    path = f"../data/oasim_param/{item}1.txt"
    df = pd.read_table(path, sep="\t", dtype={0: int})
    df.columns = ("wavelength", "absorption", "scattering")
    da = df.set_index("wavelength").to_xarray().expand_dims("component")
    da["component"] = [item]
    ds.append(da)
ds = xr.concat(ds, "component")
(
    ds.hvplot.line(x="wavelength", y="absorption", by="component")
    + ds.hvplot.line(x="wavelength", y="scattering", by="component")
).cols(1)

FileNotFoundError: [Errno 2] No such file or directory: '../data/oasim_param/dia1.txt'

The NOBM data provided by Cecile contains the ocean constituents that are sufficient inputs for the OASIM Fortran library to calculte Rrs.

Below 350nm however, there is no phytoplankton absorption data so those Rrs values should be ignored.

In [ ]:
paths = [
    f"../data/rrs_day/rrs{1998+i}{1+j:02}.nc" for j in range(12) for i in range(24)
]
ds = xr.open_mfdataset(paths)

In [ ]:
# FIXME move to re_nobm_pcc.simulate.py once oasim is fixed
ds = ds.roll({"lon": ds.sizes["lon"] // 2})
coords = xr.Dataset(
    coords={
        "lon": np.linspace(-180, 180, ds.sizes["lon"]),
        "lat": np.linspace(-84, 71.4, ds.sizes["lat"]),
    },
)
ds = xr.merge((ds.drop_vars(("lon", "lat")), coords))

In [ ]:
class Dashboard(p.Parameterized):
    # part of the GUI
    date = p.Date(dt.date(1998, 1, 1))
    h2o = p.Selector(
        ["tot", "dtc", "pic", "cdc", "t", "s"], label="Ocean Property Variable"
    )
    phy = p.Selector(ds["component"].values.tolist(), label="Phytoplankton Group")
    # needed as dependencies, not part of the GUI
    data = p.ClassSelector(xr.Dataset)
    stream = hv.streams.Tap(x=0, y=0)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.tap = xr.DataArray(
            np.empty((ds.sizes["wavelength"], 0), dtype=ds["rrs"].dtype),
            dims=("wavelength", "tap"),
            name="rrs",
        )

    @p.depends("date", watch=True, on_init=True)
    def _load_date(self):
        self.data = ds.sel({"date": np.datetime64(self.date)}).load()

    @p.depends("data", "h2o")
    def plt_h2o(self):
        da = self.data[self.h2o]
        return da.hvplot.image(x="lon", y="lat", clabel=self.h2o, title="")

    @p.depends("data", "phy")
    def plt_phy(self):
        da = self.data["phy"].sel({"component": self.phy})
        plt = da.hvplot.image(x="lon", y="lat", clabel=self.phy, title="")
        self.stream.source = plt
        return plt

    @p.depends("stream.x", "stream.y")
    def plt_rrs(self):
        da = self.data["rrs"].sel(
            {"lon": self.stream.x, "lat": self.stream.y},
            method="nearest",
        )
        da = da.expand_dims("tap")
        self.tap = xr.concat((self.tap, da), dim="tap")
        return self.tap.hvplot(x="wavelength", by="tap", title="")


dash = Dashboard(name="NOBM Variables and Computed Rrs")
pn.Column(
    pn.panel(
        dash.param,
        parameters=["date", "h2o", "phy"],
        widgets={"date": pn.widgets.DatePicker},
    ),
    dash.plt_h2o,
    dash.plt_phy,
    dash.plt_rrs,
)

## Chl-a OC4 Algorithm

OC4 (SeaWiFS) from https://oceancolor.gsfc.nasa.gov/atbd/chlor_a/

In [ ]:
a = [0.32814, -3.20725, 3.22969, -1.36769, -0.81739]
blue = [443, 489, 510]
green = 555

dim = "wavelength"
da = xr.DataArray(
    np.arange(len(WAVELENGTH)),
    coords={dim: ds[dim].loc[WAVELENGTH[0] : WAVELENGTH[-1]]},
)
blue = da.sel({dim: blue}, method="nearest").values.tolist()
green = da.sel({dim: green}, method="nearest").values.item()

a = tf.expand_dims(tf.constant(a), 1)
blue, green

In [ ]:
@tf.function
def log_blue_green_ratio(x, y):
    return (
        tf.expand_dims(
            tf.experimental.numpy.log10(
                tf.reduce_max(tf.gather(x, blue, axis=1), axis=1) / x[:, green]
            ),
            axis=1,
        ),
        tf.expand_dims(tf.experimental.numpy.log10(tf.reduce_sum(y, axis=1)), axis=1),
    )


batch_size = 2**10
tfds_rrs_day = tfds.builder("rrs_day_tfds", data_dir=DATA_DIR)
train, test = tfds_rrs_day.as_dataset(
    split=["split[:8%]", "split[9%:10%]"], as_supervised=True
)
train_size = train.cardinality()
test_size = test.cardinality()
train = train.batch(batch_size).cache()
test = test.batch(batch_size).cache()

### no retraining

In [ ]:
log_x, log_y = test.map(log_blue_green_ratio).rebatch(test_size).get_single_element()
log_y = log_y[:, 0].numpy()
log_y_hat = (log_x ** tf.range(5, dtype=np.float32) @ a)[:, 0].numpy()

In [ ]:
R2 = 1 - ((log_y - log_y_hat) ** 2).sum() / ((log_y - log_y.mean()) ** 2).sum()
print(f"R2: {R2}")

In [ ]:
(
    hv.Scatter(
        (log_y_hat[: 2**16], log_y[: 2**16]),
        kdims="prediction",
        vdims="truth",
    )
    + hv.HexTiles(
        (log_y_hat, log_y),
        kdims=["prediction", "truth"],
    ).opts(logz=True)
) * hv.Slope(1, 0).opts(color="black", line_width=1)

### re-trained

In [ ]:
network = tf.keras.Sequential(
    [
        tf.keras.layers.Lambda(lambda x: x ** tf.range(1, 5, dtype=np.float32)),
        tf.keras.layers.Dense(1),
    ]
)
network.compile(
    optimizer=tf.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.MeanSquaredError(),
)

In [ ]:
fit = network.fit(
    train.map(log_blue_green_ratio).shuffle(batch_size * 4),
    epochs=10,
)

In [ ]:
network.layers[1].weights

In [ ]:
log_x, log_y = test.map(log_blue_green_ratio).rebatch(test_size).get_single_element()
log_y_hat = network(log_x)[:, 0].numpy()
log_y = log_y[:, 0].numpy()

In [ ]:
R2 = 1 - ((log_y - log_y_hat) ** 2).sum() / ((log_y - log_y.mean()) ** 2).sum()
print(f"R2: {R2}")

In [ ]:
(
    hv.Scatter(
        (log_y_hat[: 2**16], log_y[: 2**16]),
        kdims="prediction",
        vdims="truth",
    )
    + hv.HexTiles(
        (log_y_hat, log_y),
        kdims=["prediction", "truth"],
    ).opts(logz=True)
) * hv.Slope(1, 0).opts(color="black", line_width=1)

## mlp

In [ ]:
@tf.function
def four_wavelengths(x, y):
    return (
        tf.experimental.numpy.log10(
            tf.concat(
                (tf.gather(x, blue, axis=1), tf.gather(x, [green], axis=1)),
                axis=1,
            )
        ),
        tf.experimental.numpy.log10(tf.reduce_sum(y, axis=1)),
    )


network = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(1),
    ]
)
network.compile(
    optimizer=tf.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.MeanSquaredError(),
)

In [ ]:
fit = network.fit(
    train.map(four_wavelengths).shuffle(batch_size * 4),
    epochs=10,
)

In [ ]:
log_x, log_y = test.map(four_wavelengths).rebatch(test_size).get_single_element()
log_y_hat = network(log_x)[:, 0].numpy()
log_y = log_y.numpy()

In [ ]:
R2 = 1 - ((log_y - log_y_hat) ** 2).sum() / ((log_y - log_y.mean()) ** 2).sum()
print(f"R2: {R2}")

In [ ]:
(
    hv.Scatter(
        (log_y_hat[: 2**16], log_y[: 2**16]),
        kdims="prediction",
        vdims="truth",
    )
    + hv.HexTiles(
        (log_y_hat, log_y),
        kdims=["prediction", "truth"],
    ).opts(logz=True)
) * hv.Slope(1, 0).opts(color="black", line_width=1)

## mlp, loc scale out

In [ ]:
network = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(64, "relu"),
        tf.keras.layers.Dense(2),
        tfp.layers.IndependentNormal(1),
    ]
)
network.compile(
    optimizer=tf.optimizers.Adam(learning_rate=3e-4),
    loss=lambda y, model: tf.reduce_sum(-model.log_prob(y)),
)

In [ ]:
fit = network.fit(
    train.map(four_wavelengths).shuffle(batch_size * 4),
    epochs=10,
)

In [ ]:
log_x, log_y = test.map(four_wavelengths).rebatch(test_size).get_single_element()
log_y_model = network(log_x)

In [ ]:
idx = log_y_model.stddev() / tf.abs(log_y_model.mean()) < 0.5

In [ ]:
log_y_hat = tf.boolean_mask(log_y_model.sample(), idx).numpy()
log_y = tf.boolean_mask(log_y, idx[:, 0]).numpy()

In [ ]:
R2 = 1 - ((log_y - log_y_hat) ** 2).sum() / ((log_y - log_y.mean()) ** 2).sum()
print(f"R2: {R2}")

In [ ]:
(
    hv.Scatter(
        (log_y_hat[: 2**16], log_y[: 2**16]),
        kdims="prediction",
        vdims="truth",
    )
    + hv.HexTiles(
        (log_y_hat, log_y),
        kdims=["prediction", "truth"],
    ).opts(logz=True)
) * hv.Slope(1, 0).opts(color="black", line_width=1)

In [ ]:
# HERE transform to standard normal and plot ecdf

## Spectum with Taxa

In [ ]:
x, y = next(train.shuffle(32).as_numpy_iterator())
(hv.Curve(x) + hv.Bars(y)).opts(shared_axes=False)

---

## Outdated Below

## Preprocessed Data

The features and labels are both model output from NASA GMAO using the [NOBM and OASIM](https://gmao.gsfc.nasa.gov/gmaoftp/NOBM) models. The labels are four phytoplankton chlorophyll densities output by NOBM. The features are normalized water leaving radiances output by OASIM, using the NOBM model as input.

### Features

One NetCDF file contains all the predictor data. Note that the `FillValue` attribute is not set to `9.99e11` in the netCDF file (Cecile will fix in next version). There are no explicit coordinates given; they are documented as attributes.

## Labels

Each of twelve NetCDF files contain a month of NOBM model output. The first is representative. Unlike the HyperLwn file, this one contains coordinates.

The `PhytoChl` xarray.Dataset includes the different phytoplankton groups as variables.

## Plot your Data

## Features

The radiances currently make a nice map, but the data should be more sparsely sampled.

A few "typical" hyperspectral radiances.

Mean centered radiances and corresponding phytoplankton abundances.

SVD to reduce the wavelength dimension to `k` vectors accounting for the most variation in the features. The singular values are:

The corresponding vectors:

A matrix of univariate (diagonal) and bivariate (off-diagonal) histograms of the `scores`, or coefficients generating each wavelength by linear combination of the `vectors` above.

## Labels

A map of the phytoplankton labels in `PhytoChl` at one month.

The distribution of the four phytoplankton groups.